In [1]:
import arcpy

# Access current ArcGIS Pro project and map
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Input layer name (update if different in the map)
input_layer_name = "CAMSCenterlines"  # replace if the name differs in your map
output_layer_name = "CAMSCenterlines_Dissolved"

# Check if the layer exists in the map
input_layer = None
for lyr in map_.listLayers():
    if lyr.name == input_layer_name:
        input_layer = lyr
        break

if not input_layer:
    raise ValueError(f"Layer '{input_layer_name}' not found in the current map.")

# Get layer path from the source
input_path = input_layer.dataSource

# Get the geodatabase or folder path
if input_path.endswith(".shp"):
    output_path = input_path.replace(".shp", "_Dissolved.shp")
else:
    workspace = arcpy.env.workspace = arcpy.Describe(input_path).path
    output_path = arcpy.Describe(workspace).catalogPath + f"\\{output_layer_name}"

# Perform dissolve by FullName
arcpy.management.Dissolve(
    in_features=input_path,
    out_feature_class=output_path,
    dissolve_field="FullName",
    statistics_fields="",
    multi_part="SINGLE_PART",  # or "MULTI_PART" if you want multi-part lines
    unsplit_lines="DISSOLVE_LINES"
)

print(f"Dissolved shapefile created at: {output_path}")


Dissolved shapefile created at: C:\Users\JLin\Downloads\CAMSCenterlines (1)\CAMSCenterlines_Dissolved.shp


In [2]:
import arcpy
import difflib

# Configuration
SHAPEFILE = "CAMSCenterlines_Dissolved"  # Your street centerlines shapefile
SHAPEFILE_FIELD = "FullName"  # Only field to match from shapefile
TABLE = "Section List"  # Your table
TABLE_FIELDS = ["Name", "From_", "To"]  # Fields to match from table
THRESHOLD = 0.70  # Fuzzy matching threshold (0-1)

# Street suffix equivalence table
SUFFIX_EQUIVALENCE = {
    'st': ['st', 'street', 'str'],
    'ave': ['ave', 'avenue', 'av'],
    'rd': ['rd', 'road'],
    'dr': ['dr', 'drive', 'drv'],
    'blvd': ['blvd', 'boulevard', 'bvd'],
    'ln': ['ln', 'lane'],
    'ct': ['ct', 'court', 'crt'],
    'cir': ['cir', 'circle'],
    'pl': ['pl', 'place'],
    'way': ['way', 'wy']
}

def normalize_name(name):
    """Normalize names by standardizing suffixes and removing punctuation"""
    if name is None:
        return ""
    
    name = str(name).lower().strip()
    name = name.replace('.', '').replace(',', '')
    
    parts = name.split()
    if len(parts) >= 2:
        base = ' '.join(parts[:-1])
        suffix = parts[-1]
        
        # Find matching suffix group
        for primary, variants in SUFFIX_EQUIVALENCE.items():
            if suffix in variants:
                return f"{base} {primary}"
    
    return name

def are_suffixes_equivalent(suffix1, suffix2):
    """Check if two suffixes are equivalent"""
    suffix1 = suffix1.lower().strip()
    suffix2 = suffix2.lower().strip()
    
    for primary, variants in SUFFIX_EQUIVALENCE.items():
        if suffix1 in variants and suffix2 in variants:
            return True
    return False

def fuzzy_match(target, choices):
    """Enhanced fuzzy matching with suffix handling"""
    best_match = None
    best_score = 0
    
    target_norm = normalize_name(target)
    target_parts = target_norm.split()
    
    for choice in choices:
        if choice is None:
            continue
            
        choice_norm = normalize_name(choice)
        choice_parts = choice_norm.split()
        
        # Handle suffix comparison
        if len(target_parts) >= 2 and len(choice_parts) >= 2:
            target_base = ' '.join(target_parts[:-1])
            target_suffix = target_parts[-1]
            choice_base = ' '.join(choice_parts[:-1])
            choice_suffix = choice_parts[-1]
            
            base_score = difflib.SequenceMatcher(None, target_base, choice_base).ratio()
            suffix_score = 1.0 if are_suffixes_equivalent(target_suffix, choice_suffix) else 0.0
            score = (base_score * 0.85) + (suffix_score * 0.15)
        else:
            score = difflib.SequenceMatcher(None, target_norm, choice_norm).ratio()
        
        if score > best_score:
            best_score = score
            best_match = choice
    
    return (best_match, best_score) if best_score >= THRESHOLD else (None, best_score)

def main():
    # Collect all street names from shapefile
    shapefile_names = set()
    with arcpy.da.SearchCursor(SHAPEFILE, [SHAPEFILE_FIELD]) as cursor:
        for row in cursor:
            if row[0]:
                shapefile_names.add(str(row[0]).strip())
    
    # Collect all values from table
    table_values = set()
    with arcpy.da.SearchCursor(TABLE, TABLE_FIELDS) as cursor:
        for row in cursor:
            for value in row:
                if value:
                    table_values.add(str(value).strip())
    
    # Convert to lists
    shapefile_list = list(shapefile_names)
    table_list = list(table_values)
    
    # Perform matching
    matches = {}
    for table_value in table_list:
        best_match, score = fuzzy_match(table_value, shapefile_list)
        if best_match:
            matches[table_value] = (best_match, score)
    
    # Print results
    print("Fuzzy Matching Results:")
    print("=" * 50)
    for table_val, (shape_val, score) in sorted(matches.items(), key=lambda x: -x[1][1]):
        print(f"TABLE: {table_val.ljust(30)} → SHAPEFILE: {shape_val.ljust(30)} (Score: {score:.2f})")
    
    # Show unmatched values
    unmatched = set(table_list) - set(matches.keys())
    if unmatched:
        print("\nNo matches found for:")
        for val in sorted(unmatched):
            print(f"- {val}")

if __name__ == "__main__":
    main()

Fuzzy Matching Results:
TABLE: Cabana Ave                     → SHAPEFILE: Cabana Ave                     (Score: 1.00)
TABLE: Greycliff Ave.                 → SHAPEFILE: Greycliff Ave                  (Score: 1.00)
TABLE: Elliot Ave.                    → SHAPEFILE: Elliot Ave                     (Score: 1.00)
TABLE: Central Ave.                   → SHAPEFILE: Central Ave                    (Score: 1.00)
TABLE: Hudson Ave.                    → SHAPEFILE: Hudson Ave                     (Score: 1.00)
TABLE: Evanwood Ave.                  → SHAPEFILE: Evanwood Ave                   (Score: 1.00)
TABLE: Aldgate Ave                    → SHAPEFILE: Aldgate Ave                    (Score: 1.00)
TABLE: Leverett Ave.                  → SHAPEFILE: Leverett Ave                   (Score: 1.00)
TABLE: Roundabout Dr.                 → SHAPEFILE: Roundabout Dr                  (Score: 1.00)
TABLE: Culp St.                       → SHAPEFILE: Culp St                        (Score: 1.00)
TABLE: Nevers St

TABLE: Appleblossom St.               → SHAPEFILE: Appleblossom St                (Score: 1.00)
TABLE: Ardilla Ave                    → SHAPEFILE: Ardilla Ave                    (Score: 1.00)
TABLE: Hacienda Blvd.                 → SHAPEFILE: Hacienda Blvd                  (Score: 1.00)
TABLE: Bluebonnet St.                 → SHAPEFILE: Bluebonnet St                  (Score: 1.00)
TABLE: Ballista Ave.                  → SHAPEFILE: Ballista Ave                   (Score: 1.00)
TABLE: Mayland Ave.                   → SHAPEFILE: Mayland Ave                    (Score: 1.00)
TABLE: Bromar St.                     → SHAPEFILE: Bromar St                      (Score: 1.00)
TABLE: Hurley St.                     → SHAPEFILE: Hurley St                      (Score: 1.00)
TABLE: Las Vecinas Dr.                → SHAPEFILE: Las Vecinas Dr                 (Score: 1.00)
TABLE: Giordano St.                   → SHAPEFILE: Giordano St                    (Score: 1.00)
TABLE: Melham Ave.                    → 

TABLE: AL E/Hacienda Blvd.            → SHAPEFILE: N Hacienda Blvd                (Score: 0.82)
TABLE: 90 E/Hacienda Blvd.            → SHAPEFILE: N Hacienda Blvd                (Score: 0.82)
TABLE: AL N/Victoria Ave.             → SHAPEFILE: E Victoria Ave                 (Score: 0.82)
TABLE: AL W/Hacienda Blvd.            → SHAPEFILE: N Hacienda Blvd                (Score: 0.82)
TABLE: AL N/Fairgrove Ave.            → SHAPEFILE: Fairgrove Ave                  (Score: 0.82)
TABLE: 0 E/Tonopah Ave.               → SHAPEFILE: Tonopah Ave                    (Score: 0.81)
TABLE: 0 W/Tonopah Ave.               → SHAPEFILE: Tonopah Ave                    (Score: 0.81)
TABLE: Evanwood                       → SHAPEFILE: Evanwood Ave                   (Score: 0.80)
TABLE: AL E/Glendora Ave.             → SHAPEFILE: Glendora Ave                   (Score: 0.80)
TABLE: AL N/Rowland St.               → SHAPEFILE: E Rowland St                   (Score: 0.80)
TABLE: AL N/San Jose Ave.             → 

In [3]:
import arcpy
import difflib
import os

# Configuration
SHAPEFILE = "CAMSCenterlines_Dissolved"
SHAPEFILE_FIELD = "FullName"
TABLE = "Section List"
TABLE_FIELDS = ["Name", "From_", "To"]
OUTPUT_NAME = "FirstRow_Matched_Segments"
THRESHOLD = 0.70

# Optional: Create in the same GDB as input shapefile
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap
workspace = None
for lyr in map_.listLayers():
    if lyr.name == SHAPEFILE:
        workspace = os.path.dirname(arcpy.Describe(lyr.dataSource).catalogPath)
        break
if not workspace:
    raise RuntimeError("Could not determine workspace from shapefile.")
arcpy.env.workspace = workspace

# ────────────────────────────────────────────────────────────
# Suffix normalization table
SUFFIX_EQUIVALENCE = {
    'st': ['st', 'street', 'str'],
    'ave': ['ave', 'avenue', 'av'],
    'rd': ['rd', 'road'],
    'dr': ['dr', 'drive', 'drv'],
    'blvd': ['blvd', 'boulevard', 'bvd'],
    'ln': ['ln', 'lane'],
    'ct': ['ct', 'court', 'crt'],
    'cir': ['cir', 'circle'],
    'pl': ['pl', 'place'],
    'way': ['way', 'wy']
}

def normalize_name(name):
    if name is None:
        return ""
    name = str(name).lower().replace('.', '').replace(',', '').strip()
    parts = name.split()
    if len(parts) >= 2:
        base, suffix = ' '.join(parts[:-1]), parts[-1]
        for primary, variants in SUFFIX_EQUIVALENCE.items():
            if suffix in variants:
                return f"{base} {primary}"
    return name

def are_suffixes_equivalent(s1, s2):
    s1, s2 = s1.lower(), s2.lower()
    for _, variants in SUFFIX_EQUIVALENCE.items():
        if s1 in variants and s2 in variants:
            return True
    return False

def fuzzy_match(target, choices):
    best_match, best_score = None, 0
    target_norm = normalize_name(target)
    t_parts = target_norm.split()
    for choice in choices:
        if not choice:
            continue
        c_norm = normalize_name(choice)
        c_parts = c_norm.split()
        if len(t_parts) >= 2 and len(c_parts) >= 2:
            t_base, t_sfx = ' '.join(t_parts[:-1]), t_parts[-1]
            c_base, c_sfx = ' '.join(c_parts[:-1]), c_parts[-1]
            base_score = difflib.SequenceMatcher(None, t_base, c_base).ratio()
            suffix_score = 1.0 if are_suffixes_equivalent(t_sfx, c_sfx) else 0.0
            score = base_score * 0.85 + suffix_score * 0.15
        else:
            score = difflib.SequenceMatcher(None, target_norm, c_norm).ratio()
        if score > best_score:
            best_match, best_score = choice, score
    return best_match if best_score >= THRESHOLD else None

# ────────────────────────────────────────────────────────────
# Main logic
def main():
    print("Getting first row from table...")
    with arcpy.da.SearchCursor(TABLE, TABLE_FIELDS) as cursor:
        first_row = next(cursor)
    
    # Collect shapefile names
    print("Collecting all street names from shapefile...")
    shapefile_names = set()
    with arcpy.da.SearchCursor(SHAPEFILE, [SHAPEFILE_FIELD]) as cursor:
        for row in cursor:
            shapefile_names.add(str(row[0]).strip())

    search_targets = list(first_row)
    matched_names = set()

    for target in search_targets:
        if not target:
            continue
        match = fuzzy_match(target, shapefile_names)
        if match:
            matched_names.add(match)

    if not matched_names:
        print("No matches found.")
        return

    # Build SQL expression
    sql_parts = []
    field = arcpy.AddFieldDelimiters(SHAPEFILE, SHAPEFILE_FIELD)
    for val in matched_names:
        safe_val = val.replace("'", "''")  # Escape single quotes
        sql_parts.append(f"{field} = '{safe_val}'")

    sql_expression = " OR ".join(sql_parts)

    # Select and export
    print("Selecting matched features...")
    selection = arcpy.management.SelectLayerByAttribute(SHAPEFILE, "NEW_SELECTION", sql_expression)
    output_path = os.path.join(workspace, f"{OUTPUT_NAME}.shp")

    print(f"Exporting matched features to {output_path}...")
    arcpy.management.CopyFeatures(selection, output_path)

    print(f"\n✅ Done! Output shapefile with matched lines saved to:\n{output_path}")

if __name__ == "__main__":
    main()


Getting first row from table...
Selecting matched features...
Exporting matched features to C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Matched_Segments.shp...

✅ Done! Output shapefile with matched lines saved to:
C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Matched_Segments.shp


In [4]:
import arcpy
import os

# Config
input_layer_name = "FirstRow_Matched_Segments"
output_name = "FirstRow_Intersections"
name_field = "FullName"  # Field containing street names
tolerance = 0.1  # Distance tolerance for considering points as duplicates (in map units)

# Get current ArcGIS Pro project and map
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

# Find input layer in the map and get its path
input_layer = None
for lyr in map_.listLayers():
    if lyr.name == input_layer_name:
        input_layer = lyr
        break

if not input_layer:
    raise ValueError(f"Layer '{input_layer_name}' not found in map.")

input_path = arcpy.Describe(input_layer).catalogPath
workspace = os.path.dirname(input_path)
arcpy.env.workspace = workspace

# Output path
output_path = os.path.join(workspace, f"{output_name}.shp")

# Check if layer has more than one feature
feature_count = int(arcpy.management.GetCount(input_path)[0])
if feature_count < 2:
    raise ValueError("Intersection requires at least 2 line features. Only found one.")

def remove_duplicate_points(points_data, tolerance):
    """
    Remove duplicate intersection points within tolerance distance
    points_data: list of tuples (point_geometry, street1, street2, intersection_name)
    tolerance: distance tolerance for considering points as duplicates
    """
    if not points_data:
        return []
    
    unique_points = []
    
    for point_geom, street1, street2, int_name in points_data:
        is_duplicate = False
        
        # Check against existing unique points
        for existing_geom, existing_street1, existing_street2, existing_name in unique_points:
            # Calculate distance between points
            distance = point_geom.distanceTo(existing_geom)
            
            if distance <= tolerance:
                # Points are close enough to be considered duplicates
                # Check if it's the same intersection (same streets but possibly different order)
                streets_set1 = {street1.lower(), street2.lower()}
                streets_set2 = {existing_street1.lower(), existing_street2.lower()}
                
                if streets_set1 == streets_set2:
                    is_duplicate = True
                    break
        
        if not is_duplicate:
            unique_points.append((point_geom, street1, street2, int_name))
    
    return unique_points

# Run intersection - Method 1: Specify same input twice for self-intersection
print(f"Finding intersections for '{input_layer_name}'...")

try:
    # Method 1: Self-intersection by specifying input twice with ALL attributes
    arcpy.analysis.Intersect(
        in_features=[input_path, input_path],
        out_feature_class=output_path,
        join_attributes="ALL",  # Keep all attributes to get street names
        cluster_tolerance="",
        output_type="POINT"
    )
    
    # Add fields for intersecting street names if they don't exist
    # Note: Shapefile field names are limited to 10 characters
    field_names = [f.name for f in arcpy.ListFields(output_path)]
    if "Street1" not in field_names:
        arcpy.management.AddField(output_path, "Street1", "TEXT", field_length=50)
    if "Street2" not in field_names:
        arcpy.management.AddField(output_path, "Street2", "TEXT", field_length=50)
    if "IntName" not in field_names:  # Fixed: was "Intersection" (too long for shapefile)
        arcpy.management.AddField(output_path, "IntName", "TEXT", field_length=100)
    
    # Populate the street name fields and collect points for deduplication
    name_fields = [f for f in field_names if name_field in f]
    points_to_keep = []
    
    if len(name_fields) >= 2:
        with arcpy.da.SearchCursor(output_path, ["SHAPE@"] + name_fields) as cursor:
            for row in cursor:
                point_geom = row[0]
                street1 = row[1] if row[1] else ""
                street2 = row[2] if row[2] else ""
                
                # Skip if both streets are the same (self-intersection)
                if street1.lower() != street2.lower():
                    int_name = f"{street1} & {street2}"
                    points_to_keep.append((point_geom, street1, street2, int_name))
    
    # Remove duplicates
    unique_points = remove_duplicate_points(points_to_keep, tolerance)
    
    # Delete all records and insert unique ones
    arcpy.management.DeleteRows(output_path)
    
    if unique_points:
        with arcpy.da.InsertCursor(output_path, ["SHAPE@", "Street1", "Street2", "IntName"]) as cursor:
            for point_geom, street1, street2, int_name in unique_points:
                cursor.insertRow([point_geom, street1, street2, int_name])
    
    print(f"\n✅ Done! Intersections saved to: {output_path}")
    print(f"Found {len(unique_points)} unique intersection points")
    
except Exception as e:
    print(f"Method 1 failed: {e}")
    print("Trying manual intersection detection with street names...")
    
    # Method 2: Manual approach using cursors with street names
    print("Using manual intersection detection...")
    
    # Create output feature class with custom fields
    spatial_ref = arcpy.Describe(input_path).spatialReference
    arcpy.management.CreateFeatureclass(
        out_path=workspace,
        out_name=os.path.basename(output_path).replace('.shp', ''),
        geometry_type="POINT",
        spatial_reference=spatial_ref
    )
    
    # Add fields for street names (shapefile field names max 10 chars)
    arcpy.management.AddField(output_path, "Street1", "TEXT", field_length=50)
    arcpy.management.AddField(output_path, "Street2", "TEXT", field_length=50)
    arcpy.management.AddField(output_path, "IntName", "TEXT", field_length=100)
    
    # Get all line geometries with their names
    lines_data = []
    with arcpy.da.SearchCursor(input_path, ["SHAPE@", name_field]) as cursor:
        for row in cursor:
            lines_data.append((row[0], row[1] if row[1] else "Unknown"))
    
    # Find intersections between different lines
    intersections_data = []
    for i in range(len(lines_data)):
        for j in range(i + 1, len(lines_data)):
            geom1, name1 = lines_data[i]
            geom2, name2 = lines_data[j]
            
            # Only find intersections between different street names
            if name1.lower() != name2.lower():
                intersection = geom1.intersect(geom2, 1)  # 1 = point dimension
                if not intersection.isEmpty():
                    intersection_name = f"{name1} & {name2}"
                    
                    if intersection.type == "point":
                        intersections_data.append((intersection, name1, name2, intersection_name))
                    elif intersection.type == "multipoint":
                        # Handle multipoint geometries
                        for part in intersection:
                            for point in part:
                                point_geom = arcpy.PointGeometry(point, spatial_ref)
                                intersections_data.append((point_geom, name1, name2, intersection_name))
    
    # Remove duplicates from intersections
    unique_intersections = remove_duplicate_points(intersections_data, tolerance)
    
    # Insert unique intersection points with street names
    if unique_intersections:
        with arcpy.da.InsertCursor(output_path, ["SHAPE@", "Street1", "Street2", "IntName"]) as cursor:
            for intersection_geom, street1, street2, intersection_name in unique_intersections:
                cursor.insertRow([intersection_geom, street1, street2, intersection_name])
    
    print(f"\n✅ Done! Intersections saved to: {output_path}")
    print(f"Found {len(unique_intersections)} unique intersection points with street names")

# Optional: Add index on intersection name for better performance
try:
    arcpy.management.AddIndex(output_path, ["IntName"], "IntName_idx")
    print("Added index on intersection names")
except:
    pass  # Index creation might fail if it already exists

Finding intersections for 'FirstRow_Matched_Segments'...

✅ Done! Intersections saved to: C:\Users\JLin\Downloads\CAMSCenterlines (1)\FirstRow_Intersections.shp
Found 3 unique intersection points


In [5]:
import arcpy
import os
from collections import defaultdict

# Config
input_lines_layer = "FirstRow_Matched_Segments"  # Original line segments
intersections_layer = "FirstRow_Intersections"   # Intersection points we created
standalone_table = "Section List"               # Name of your standalone table
output_segments = "Intersection_Segments"        # Output segments between intersections
output_street2_segments = "Street2_Segments"     # New output for Street2-based segments
name_field = "FullName"                         # Field containing street names
table_match_field = "Name"                # Field in table to match against
tolerance = 1.0                                 # Tolerance for spatial operations

# Get current ArcGIS Pro project and map
project = arcpy.mp.ArcGISProject("CURRENT")
map_ = project.activeMap

def find_layer_by_name(layer_name):
    """Find layer in map by name"""
    for lyr in map_.listLayers():
        if lyr.name == layer_name:
            return lyr
    return None

def find_table_by_name(table_name):
    """Find table in map by name"""
    for table in map_.listTables():
        if table.name == table_name:
            return table
    return None

# Find input layers
lines_layer = find_layer_by_name(input_lines_layer)
intersections_layer_obj = find_layer_by_name(intersections_layer)
table_obj = find_table_by_name(standalone_table)

if not lines_layer:
    raise ValueError(f"Lines layer '{input_lines_layer}' not found in map.")
if not intersections_layer_obj:
    raise ValueError(f"Intersections layer '{intersections_layer}' not found in map.")
if not table_obj:
    raise ValueError(f"Table '{standalone_table}' not found in map.")

# Get paths
lines_path = arcpy.Describe(lines_layer).catalogPath
intersections_path = arcpy.Describe(intersections_layer_obj).catalogPath
table_path = arcpy.Describe(table_obj).catalogPath
workspace = os.path.dirname(lines_path)
arcpy.env.workspace = workspace

# Output path
output_path = os.path.join(workspace, f"{output_segments}.shp")

print(f"Processing line segments between intersections...")
print(f"Lines: {lines_path}")
print(f"Intersections: {intersections_path}")
print(f"Table: {table_path}")

def get_intersections_by_street(intersections_path):
    """Get all intersection points organized by street name"""
    street_intersections = defaultdict(list)
    
    with arcpy.da.SearchCursor(intersections_path, ["SHAPE@", "Street1", "Street2", "IntName"]) as cursor:
        for row in cursor:
            point_geom, street1, street2, int_name = row
            # Add point to both streets it intersects
            street_intersections[street1.lower()].append({
                'geometry': point_geom,
                'street1': street1,
                'street2': street2,
                'int_name': int_name
            })
            street_intersections[street2.lower()].append({
                'geometry': point_geom,
                'street1': street1,
                'street2': street2,
                'int_name': int_name
            })
    
    return street_intersections

def get_table_records(table_path, match_field):
    """Get all records from standalone table"""
    table_records = {}
    
    # Get all field names from table
    field_names = [f.name for f in arcpy.ListFields(table_path)]
    
    with arcpy.da.SearchCursor(table_path, field_names) as cursor:
        for row in cursor:
            # Create dictionary of field:value pairs
            record = dict(zip(field_names, row))
            street_name = record.get(match_field, "").lower()
            if street_name:
                if street_name not in table_records:
                    table_records[street_name] = []
                table_records[street_name].append(record)
    
    return table_records

def split_line_at_points(line_geom, intersection_points, tolerance):
    """Split a line geometry at intersection points"""
    if not intersection_points:
        return [line_geom]
    
    # Sort intersection points along the line
    points_with_measure = []
    for point_info in intersection_points:
        point_geom = point_info['geometry']
        
        # Extract the actual point from the geometry
        if hasattr(point_geom, 'firstPoint'):
            actual_point = point_geom.firstPoint
        else:
            actual_point = point_geom
        
        # Get the measure (position along line) for this point
        try:
            measure = line_geom.measureOnLine(actual_point)
            points_with_measure.append((measure, point_info, actual_point))
        except Exception as e:
            print(f"Warning: Could not measure point on line: {e}")
            continue
    
    # Sort by measure (position along line)
    points_with_measure.sort(key=lambda x: x[0])
    
    # If we have valid points to split at, use a different approach
    if not points_with_measure:
        return [line_geom]
    
    # Use a simpler approach - split the line using segmentAlongLine
    segments = []
    
    # Create segments between consecutive intersection points
    measures = [0] + [p[0] for p in points_with_measure] + [line_geom.length]
    
    for i in range(len(measures) - 1):
        start_measure = measures[i]
        end_measure = measures[i + 1]
        
        # Skip if the segment would be too short
        if end_measure - start_measure < tolerance:
            continue
            
        try:
            # Create segment from start to end measure
            segment = line_geom.segmentAlongLine(start_measure, end_measure)
            if segment and hasattr(segment, 'length') and segment.length > tolerance:
                segments.append(segment)
        except Exception as e:
            print(f"Warning: Could not create segment from {start_measure} to {end_measure}: {e}")
            continue
    
    # If no segments were created, return the original line
    if not segments:
        return [line_geom]
    
    return segments

# Get intersection points organized by street
print("Organizing intersection points by street...")
street_intersections = get_intersections_by_street(intersections_path)

# Get table records
print("Loading table records...")
table_records = get_table_records(table_path, table_match_field)

# Create output feature class
spatial_ref = arcpy.Describe(lines_path).spatialReference
arcpy.management.CreateFeatureclass(
    out_path=workspace,
    out_name=os.path.basename(output_path).replace('.shp', ''),
    geometry_type="POLYLINE",
    spatial_reference=spatial_ref
)

# Add fields for segment information (keeping field names <= 10 characters for shapefile compatibility)
arcpy.management.AddField(output_path, "StreetName", "TEXT", field_length=100)
arcpy.management.AddField(output_path, "FromInt", "TEXT", field_length=150)
arcpy.management.AddField(output_path, "ToInt", "TEXT", field_length=150)
arcpy.management.AddField(output_path, "HasTableR", "TEXT", field_length=5)  # YES/NO (shortened from HasTableRec)
arcpy.management.AddField(output_path, "TableCount", "LONG")
arcpy.management.AddField(output_path, "SegmentID", "TEXT", field_length=50)
arcpy.management.AddField(output_path, "BestMatch", "TEXT", field_length=5)  # YES/NO - indicates the best matching segment
arcpy.management.AddField(output_path, "MatchScore", "DOUBLE")  # Score for segment matching

# Add fields from the original lines layer
original_fields = [f.name for f in arcpy.ListFields(lines_path) 
                   if f.type not in ['OID', 'Geometry'] and f.name.upper() not in ['SHAPE_LENGTH', 'SHAPE_AREA']]
for field_name in original_fields:
    field_obj = arcpy.ListFields(lines_path, field_name)[0]
    arcpy.management.AddField(output_path, field_name, field_obj.type, field_length=field_obj.length)

def calculate_segment_match_score(segment_geom, street_name, from_int, to_int, table_records):
    """Calculate a match score for a segment based on various criteria"""
    if not table_records:
        return 0.0
    
    score = 0.0
    
    # Base score for having table records
    score += 10.0
    
    # Length-based scoring (prefer segments of reasonable length)
    segment_length = segment_geom.length
    if 50 <= segment_length <= 500:  # Reasonable block length
        score += 20.0
    elif 20 <= segment_length <= 1000:
        score += 10.0
    
    # Intersection-based scoring (prefer segments between named intersections)
    if from_int and to_int and from_int != "START" and to_int != "END":
        score += 30.0
    elif (from_int and from_int != "START") or (to_int and to_int != "END"):
        score += 15.0
    
    # Check if table records contain location hints
    for record in table_records:
        # Look for common fields that might indicate location
        for field_name, field_value in record.items():
            if field_value and isinstance(field_value, str):
                field_value_lower = field_value.lower()
                
                # Check for intersection mentions in table data
                if from_int and from_int.lower() in field_value_lower:
                    score += 15.0
                if to_int and to_int.lower() in field_value_lower:
                    score += 15.0
                
                # Look for common location descriptors
                if any(keyword in field_value_lower for keyword in 
                       ['main', 'primary', 'central', 'downtown', 'business']):
                    score += 10.0
    
    return score

print("Processing line segments...")
segment_count = 0
matched_count = 0

def select_best_matching_segments(output_path):
    """Identify the best matching segment for each street that has table records"""
    
    # Determine the correct ID field name (FID for shapefiles, OBJECTID for geodatabase)
    id_field = "FID" if output_path.endswith('.shp') else "OBJECTID"
    
    # Group segments by street name
    street_segments = defaultdict(list)
    
    # Read all segments
    with arcpy.da.SearchCursor(output_path, [id_field, "StreetName", "HasTableR", "MatchScore"]) as cursor:
        for row in cursor:
            oid, street_name, has_table, match_score = row
            if has_table == "YES":
                street_segments[street_name].append((oid, match_score))
    
    # For each street, find the segment with the highest match score
    best_segments = {}
    for street_name, segments in street_segments.items():
        if segments:
            # Sort by match score (descending) and take the first one
            best_segment = max(segments, key=lambda x: x[1])
            best_segments[best_segment[0]] = True
    
    # Update the BestMatch field
    with arcpy.da.UpdateCursor(output_path, [id_field, "BestMatch"]) as cursor:
        for row in cursor:
            oid, _ = row
            if oid in best_segments:
                cursor.updateRow([oid, "YES"])
            else:
                cursor.updateRow([oid, "NO"])
    
    return len(best_segments)

def create_street2_segments(intersections_path, output_path, spatial_ref, table_records):
    """Create line segments connecting intersections that share the same Street2 value"""
    
    # Group intersections by Street2 value
    street2_groups = defaultdict(list)
    
    # Read all intersection points
    with arcpy.da.SearchCursor(intersections_path, ["SHAPE@", "Street1", "Street2", "IntName"]) as cursor:
        for row in cursor:
            point_geom, street1, street2, int_name = row
            if street2:  # Only process if Street2 has a value
                street2_groups[street2.lower()].append({
                    'geometry': point_geom,
                    'street1': street1,
                    'street2': street2,
                    'int_name': int_name
                })
    
    # Create output feature class for Street2 segments
    arcpy.management.CreateFeatureclass(
        out_path=os.path.dirname(output_path),
        out_name=os.path.basename(output_path).replace('.shp', ''),
        geometry_type="POLYLINE",
        spatial_reference=spatial_ref
    )
    
    # Add fields for Street2 segment information
    arcpy.management.AddField(output_path, "Street2Name", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "FromInt", "TEXT", field_length=150)
    arcpy.management.AddField(output_path, "ToInt", "TEXT", field_length=150)
    arcpy.management.AddField(output_path, "HasTableR", "TEXT", field_length=5)
    arcpy.management.AddField(output_path, "TableCount", "LONG")
    arcpy.management.AddField(output_path, "SegmentLen", "DOUBLE")
    arcpy.management.AddField(output_path, "PointCount", "LONG")
    
    segments_created = 0
    
    # Process each Street2 group
    with arcpy.da.InsertCursor(output_path, 
                               ["SHAPE@", "Street2Name", "FromInt", "ToInt", "HasTableR", 
                                "TableCount", "SegmentLen", "PointCount"]) as insert_cursor:
        
        for street2_name, intersections in street2_groups.items():
            if len(intersections) < 2:
                continue  # Need at least 2 points to make a line
            
            # Check if this street has records in the table
            has_table_record = "YES" if street2_name in table_records else "NO"
            table_count = len(table_records.get(street2_name, []))
            
            # Sort intersections to create logical segments
            # For now, we'll create segments between all consecutive pairs
            # You might want to add more sophisticated logic here based on your needs
            
            if len(intersections) == 2:
                # Simple case: exactly 2 intersections, create 1 segment
                point1 = intersections[0]
                point2 = intersections[1]
                
                # Create line geometry
                line_points = [point1['geometry'].firstPoint, point2['geometry'].firstPoint]
                line_geom = arcpy.Polyline(arcpy.Array(line_points), spatial_ref)
                
                insert_cursor.insertRow([
                    line_geom,
                    intersections[0]['street2'],  # Use original case
                    point1['int_name'],
                    point2['int_name'],
                    has_table_record,
                    table_count,
                    line_geom.length,
                    2
                ])
                segments_created += 1
                
            else:
                # Multiple intersections: create segments between consecutive pairs
                # Sort by some criteria (you might want to adjust this based on your data)
                sorted_intersections = sorted(intersections, key=lambda x: (x['geometry'].firstPoint.X, x['geometry'].firstPoint.Y))
                
                for i in range(len(sorted_intersections) - 1):
                    point1 = sorted_intersections[i]
                    point2 = sorted_intersections[i + 1]
                    
                    # Create line geometry
                    line_points = [point1['geometry'].firstPoint, point2['geometry'].firstPoint]
                    line_geom = arcpy.Polyline(arcpy.Array(line_points), spatial_ref)
                    
                    insert_cursor.insertRow([
                        line_geom,
                        sorted_intersections[0]['street2'],  # Use original case
                        point1['int_name'],
                        point2['int_name'],
                        has_table_record,
                        table_count,
                        line_geom.length,
                        len(intersections)
                    ])
                    segments_created += 1
    
    return segments_created
    """Identify the best matching segment for each street that has table records"""
    
    # Group segments by street name
    street_segments = defaultdict(list)
    
    # Read all segments
    with arcpy.da.SearchCursor(output_path, ["OBJECTID", "StreetName", "HasTableR", "MatchScore"]) as cursor:
        for row in cursor:
            oid, street_name, has_table, match_score = row
            if has_table == "YES":
                street_segments[street_name].append((oid, match_score))
    
    # For each street, find the segment with the highest match score
    best_segments = {}
    for street_name, segments in street_segments.items():
        if segments:
            # Sort by match score (descending) and take the first one
            best_segment = max(segments, key=lambda x: x[1])
            best_segments[best_segment[0]] = True
    
    # Update the BestMatch field
    with arcpy.da.UpdateCursor(output_path, ["OBJECTID", "BestMatch"]) as cursor:
        for row in cursor:
            oid, _ = row
            if oid in best_segments:
                cursor.updateRow([oid, "YES"])
            else:
                cursor.updateRow([oid, "NO"])
    
    return len(best_segments)

# Process each line in the original lines layer
with arcpy.da.SearchCursor(lines_path, ["SHAPE@", name_field] + original_fields) as lines_cursor:
    for line_row in lines_cursor:
        line_geom = line_row[0]
        street_name = line_row[1] if line_row[1] else "Unknown"
        original_attributes = line_row[2:]
        
        # Get intersection points for this street
        intersections = street_intersections.get(street_name.lower(), [])
        
        if len(intersections) < 2:
            # No intersections or only one intersection - keep the whole line
            segments = [line_geom]
            segment_info = [(None, None)]
        else:
            # Split line at intersection points
            segments = split_line_at_points(line_geom, intersections, tolerance)
            
            # For each segment, determine which intersections it connects
            segment_info = []
            for segment in segments:
                from_int = None
                to_int = None
                
                # Check which intersections are at the start and end of this segment
                start_point = segment.firstPoint
                end_point = segment.lastPoint
                
                for intersection in intersections:
                    # Extract the actual point from the intersection geometry
                    if hasattr(intersection['geometry'], 'firstPoint'):
                        int_point = intersection['geometry'].firstPoint
                    else:
                        int_point = intersection['geometry']
                    
                    # Calculate distance using proper geometry methods
                    start_distance = ((start_point.X - int_point.X)**2 + (start_point.Y - int_point.Y)**2)**0.5
                    end_distance = ((end_point.X - int_point.X)**2 + (end_point.Y - int_point.Y)**2)**0.5
                    
                    if start_distance <= tolerance:
                        from_int = intersection['int_name']
                    if end_distance <= tolerance:
                        to_int = intersection['int_name']
                
                segment_info.append((from_int, to_int))
        
        # Check if this street has records in the table
        has_table_record = "YES" if street_name.lower() in table_records else "NO"
        table_count = len(table_records.get(street_name.lower(), []))
        street_table_records = table_records.get(street_name.lower(), [])
        
        # Insert segments into output
        with arcpy.da.InsertCursor(output_path, 
                                   ["SHAPE@", "StreetName", "FromInt", "ToInt", "HasTableR", 
                                    "TableCount", "SegmentID", "BestMatch", "MatchScore"] + original_fields) as insert_cursor:
            
            for i, (segment_geom, (from_int, to_int)) in enumerate(zip(segments, segment_info)):
                segment_id = f"{street_name}_Seg_{segment_count + i + 1}"
                
                # Calculate match score for this segment
                match_score = calculate_segment_match_score(
                    segment_geom, street_name, from_int, to_int, street_table_records
                )
                
                insert_cursor.insertRow([
                    segment_geom,
                    street_name,
                    from_int or "START",
                    to_int or "END",
                    has_table_record,
                    table_count,
                    segment_id,
                    "NO",  # BestMatch - will be updated later
                    match_score
                ] + list(original_attributes))
        
        segment_count += len(segments)
        if has_table_record == "YES":
            matched_count += len(segments)

# Select the best matching segments for each street
print("\nIdentifying best matching segments...")
best_segment_count = select_best_matching_segments(output_path)

print(f"\n✅ Done! Segment analysis complete:")
print(f"Total segments created: {segment_count}")
print(f"Segments with table matches: {matched_count}")
print(f"Best matching segments identified: {best_segment_count}")
print(f"Output saved to: {output_path}")

# Create summary report
print(f"\n📊 Summary by street:")
street_summary = defaultdict(lambda: {'segments': 0, 'has_table': False, 'table_count': 0, 'best_segment': False})

with arcpy.da.SearchCursor(output_path, ["StreetName", "HasTableR", "TableCount", "BestMatch"]) as cursor:
    for row in cursor:
        street_name, has_table, table_count, best_match = row
        street_summary[street_name]['segments'] += 1
        street_summary[street_name]['has_table'] = (has_table == "YES")
        street_summary[street_name]['table_count'] = table_count
        if best_match == "YES":
            street_summary[street_name]['best_segment'] = True

for street, info in sorted(street_summary.items()):
    if info['has_table']:
        best_indicator = "⭐ BEST MATCH" if info['best_segment'] else "✅ MATCHED"
        status = f"{best_indicator}"
    else:
        status = "❌ NO MATCH"
    print(f"{street}: {info['segments']} segments, {info['table_count']} table records - {status}")

print(f"\n💡 Use the 'HasTableR' field to filter segments that have corresponding table records.")
print(f"💡 Use the 'BestMatch' field to identify the single best segment for each street.")
print(f"💡 Use the 'FromInt' and 'ToInt' fields to identify which intersections each segment connects.")
print(f"💡 Use the 'MatchScore' field to see the calculated match confidence.")

# Create Street2-based segments
print(f"\n🔗 Creating Street2-based line segments...")
street2_output_path = os.path.join(workspace, f"{output_street2_segments}.shp")
street2_segments_count = create_street2_seimport arcpy
import os
from collections import defaultdict
import difflib

def normalize_street_name(name):
    """Normalize street names for better matching"""
    if not name:
        return ""
    
    name = str(name).lower().strip()
    # Remove common suffixes and punctuation
    name = name.replace('.', '').replace(',', '')
    
    # Standardize common abbreviations
    replacements = {
        ' avenue': ' ave',
        ' street': ' st',
        ' drive': ' dr',
        ' road': ' rd',
        ' boulevard': ' blvd',
        ' lane': ' ln',
        ' court': ' ct'
    }
    
    for old, new in replacements.items():
        name = name.replace(old, new)
    
    return name

def fuzzy_match_street(target, candidates, threshold=0.8):
    """Find best matching street name from candidates"""
    target_norm = normalize_street_name(target)
    best_match = None
    best_score = 0
    
    for candidate in candidates:
        candidate_norm = normalize_street_name(candidate)
        score = difflib.SequenceMatcher(None, target_norm, candidate_norm).ratio()
        
        if score > best_score and score >= threshold:
            best_match = candidate
            best_score = score
    
    return best_match, best_score

def load_excel_segments(excel_path):
    """Load street segments from Excel file"""
    segments = []
    
    try:
        # Try to read as table first
        with arcpy.da.SearchCursor(excel_path, ["Name", "From_", "To", "Sec_ID", "Length", "Width"]) as cursor:
            for row in cursor:
                name, from_st, to_st, sec_id, length, width = row
                if name and from_st and to_st:
                    segments.append({
                        'name': str(name).strip(),
                        'from': str(from_st).strip(),
                        'to': str(to_st).strip(),
                        'sec_id': sec_id,
                        'length': length,
                        'width': width
                    })
    except:
        print("Note: Could not read Excel directly. Make sure it's imported as a table in ArcGIS.")
        print("Trying alternative method...")
        
        # Alternative: If Excel is imported as a standalone table
        table_name = "Section_List"  # Adjust this name as needed
        project = arcpy.mp.ArcGISProject("CURRENT")
        map_ = project.activeMap
        
        for table in map_.listTables():
            if table_name.lower() in table.name.lower():
                table_path = arcpy.Describe(table).catalogPath
                
                # Get field names
                field_names = [f.name for f in arcpy.ListFields(table_path)]
                print(f"Available fields: {field_names}")
                
                # Try different field name variations
                name_field = None
                from_field = None
                to_field = None
                
                for field in field_names:
                    field_lower = field.lower()
                    if 'name' in field_lower and not name_field:
                        name_field = field
                    elif field_lower in ['from', 'from_'] or 'from' in field_lower:
                        from_field = field
                    elif field_lower in ['to', 'to_'] or field_lower.endswith('to'):
                        to_field = field
                
                if name_field and from_field and to_field:
                    with arcpy.da.SearchCursor(table_path, [name_field, from_field, to_field]) as cursor:
                        for row in cursor:
                            name, from_st, to_st = row
                            if name and from_st and to_st:
                                segments.append({
                                    'name': str(name).strip(),
                                    'from': str(from_st).strip(),
                                    'to': str(to_st).strip(),
                                    'sec_id': None,
                                    'length': None,
                                    'width': None
                                })
                break
    
    print(f"Loaded {len(segments)} segments from Excel")
    return segments

def find_matching_segments(intersections_path, excel_segments, spatial_ref):
    """Find line segments that match Excel table rows"""
    
    # Load all intersection points
    intersections = []
    with arcpy.da.SearchCursor(intersections_path, ["SHAPE@", "Street1", "Street2", "IntName"]) as cursor:
        for row in cursor:
            point_geom, street1, street2, int_name = row
            intersections.append({
                'geometry': point_geom,
                'street1': street1,
                'street2': street2,
                'int_name': int_name,
                'point': point_geom.firstPoint
            })
    
    print(f"Found {len(intersections)} intersection points")
    
    # For each Excel segment, try to find matching intersection points
    matched_segments = []
    
    for excel_seg in excel_segments:
        street_name = excel_seg['name']
        from_street = excel_seg['from']
        to_street = excel_seg['to']
        
        print(f"\nLooking for: {street_name} from {from_street} to {to_street}")
        
        # Find intersection points that involve this street
        relevant_intersections = []
        for intersection in intersections:
            # Check if this intersection involves the target street
            streets_at_intersection = {
                normalize_street_name(intersection['street1']),
                normalize_street_name(intersection['street2'])
            }
            
            target_street_norm = normalize_street_name(street_name)
            
            # Use fuzzy matching for street names
            for street_at_int in [intersection['street1'], intersection['street2']]:
                match, score = fuzzy_match_street(street_name, [street_at_int])
                if match and score > 0.8:
                    relevant_intersections.append(intersection)
                    break
        
        print(f"  Found {len(relevant_intersections)} relevant intersections")
        
        if len(relevant_intersections) < 2:
            print(f"  ⚠️ Not enough intersections found for {street_name}")
            continue
        
        # Now find the specific FROM and TO intersections
        from_intersection = None
        to_intersection = None
        
        for intersection in relevant_intersections:
            # Check if this intersection involves the FROM street
            for street_at_int in [intersection['street1'], intersection['street2']]:
                from_match, from_score = fuzzy_match_street(from_street, [street_at_int])
                if from_match and from_score > 0.7:
                    from_intersection = intersection
                    print(f"  ✓ Found FROM intersection: {intersection['int_name']}")
                    break
            
            # Check if this intersection involves the TO street
            for street_at_int in [intersection['street1'], intersection['street2']]:
                to_match, to_score = fuzzy_match_street(to_street, [street_at_int])
                if to_match and to_score > 0.7:
                    to_intersection = intersection
                    print(f"  ✓ Found TO intersection: {intersection['int_name']}")
                    break
        
        # Create line segment if both intersections found
        if from_intersection and to_intersection and from_intersection != to_intersection:
            line_points = [from_intersection['point'], to_intersection['point']]
            line_geom = arcpy.Polyline(arcpy.Array(line_points), spatial_ref)
            
            matched_segments.append({
                'geometry': line_geom,
                'excel_data': excel_seg,
                'from_intersection': from_intersection,
                'to_intersection': to_intersection,
                'street_name': street_name,
                'calculated_length': line_geom.length
            })
            
            print(f"  ✅ Created segment: {from_intersection['int_name']} → {to_intersection['int_name']}")
        else:
            print(f"  ❌ Could not find both FROM and TO intersections")
            if not from_intersection:
                print(f"     Missing FROM: {from_street}")
            if not to_intersection:
                print(f"     Missing TO: {to_street}")
    
    return matched_segments

def create_matched_segments_layer(matched_segments, output_path):
    """Create output feature class with matched segments"""
    
    if not matched_segments:
        print("No matched segments to create")
        return 0
    
    # Get spatial reference from first segment
    spatial_ref = matched_segments[0]['geometry'].spatialReference
    
    # Create output feature class
    arcpy.management.CreateFeatureclass(
        out_path=os.path.dirname(output_path),
        out_name=os.path.basename(output_path).replace('.shp', ''),
        geometry_type="POLYLINE",
        spatial_reference=spatial_ref
    )
    
    # Add fields
    arcpy.management.AddField(output_path, "StreetName", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "FromInt", "TEXT", field_length=150)
    arcpy.management.AddField(output_path, "ToInt", "TEXT", field_length=150)
    arcpy.management.AddField(output_path, "FromSt", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "ToSt", "TEXT", field_length=100)
    arcpy.management.AddField(output_path, "ExcelSecID", "LONG")
    arcpy.management.AddField(output_path, "ExcelLen", "DOUBLE")
    arcpy.management.AddField(output_path, "CalcLen", "DOUBLE")
    arcpy.management.AddField(output_path, "LenDiff", "DOUBLE")
    arcpy.management.AddField(output_path, "Width", "DOUBLE")
    
    # Insert segments
    with arcpy.da.InsertCursor(output_path, 
                               ["SHAPE@", "StreetName", "FromInt", "ToInt", "FromSt", "ToSt",
                                "ExcelSecID", "ExcelLen", "CalcLen", "LenDiff", "Width"]) as cursor:
        
        for segment in matched_segments:
            excel_data = segment['excel_data']
            from_int = segment['from_intersection']
            to_int = segment['to_intersection']
            
            excel_length = excel_data.get('length', 0) or 0
            calc_length = segment['calculated_length']
            length_diff = abs(excel_length - calc_length) if excel_length > 0 else 0
            
            cursor.insertRow([
                segment['geometry'],
                segment['street_name'],
                from_int['int_name'],
                to_int['int_name'],
                excel_data['from'],
                excel_data['to'],
                excel_data.get('sec_id'),
                excel_length,
                calc_length,
                length_diff,
                excel_data.get('width')
            ])
    
    return len(matched_segments)

def main():
    """Main execution function"""
    
    # Configuration
    intersections_layer = "FirstRow_Intersections"
    excel_table = "Section List"  # Adjust this name as needed
    output_segments = "Matched_Table_Segments"
    
    # Get current project
    project = arcpy.mp.ArcGISProject("CURRENT")
    map_ = project.activeMap
    
    # Find intersections layer
    intersections_path = None
    for lyr in map_.listLayers():
        if lyr.name == intersections_layer:
            intersections_path = arcpy.Describe(lyr).catalogPath
            break
    
    if not intersections_path:
        raise ValueError(f"Intersections layer '{intersections_layer}' not found")
    
    # Get workspace and spatial reference
    workspace = os.path.dirname(intersections_path)
    spatial_ref = arcpy.Describe(intersections_path).spatialReference
    output_path = os.path.join(workspace, f"{output_segments}.shp")
    
    print(f"Processing intersections from: {intersections_path}")
    print(f"Output will be saved to: {output_path}")
    
    # Load Excel segments (you may need to adjust this part based on how the Excel is imported)
    excel_segments = load_excel_segments(excel_table)
    
    if not excel_segments:
        print("❌ No Excel segments loaded. Please check:")
        print("1. Excel file is imported as a table in ArcGIS")
        print("2. Table name matches the configuration")
        print("3. Required fields (Name, From, To) exist")
        return None
    
    # Find matching segments
    print(f"\n🔍 Searching for matching segments...")
    matched_segments = find_matching_segments(intersections_path, excel_segments, spatial_ref)
    
    if not matched_segments:
        print("❌ No matching segments found")
        return None
    
    # Create output layer
    print(f"\n📝 Creating output layer...")
    segment_count = create_matched_segments_layer(matched_segments, output_path)
    
    print(f"\n✅ Success! Created {segment_count} matched segments")
    print(f"Output saved to: {output_path}")
    
    # Summary
    print(f"\n📊 Summary:")
    print(f"Excel segments processed: {len(excel_segments)}")
    print(f"Matched segments created: {segment_count}")
    print(f"Success rate: {segment_count/len(excel_segments)*100:.1f}%")
    
    print(f"\n💡 Tips:")
    print(f"- Check 'LenDiff' field to see length differences between Excel and calculated")
    print(f"- Use 'ExcelSecID' to cross-reference with original Excel data")
    print(f"- Filter by 'StreetName' to focus on specific streets")
    
    return output_path

if __name__ == "__main__":
    main()gments(intersections_path, street2_output_path, spatial_ref, table_records)

print(f"\n✅ Street2 segments complete!")
print(f"Street2-based segments created: {street2_segments_count}")
print(f"Street2 segments saved to: {street2_output_path}")
print(f"\n💡 Street2 segments connect intersection points that share the same Street2 value.")
print(f"💡 These represent the actual street segments you highlighted in your intersection table.")

SyntaxError: invalid syntax (<string>, line 529)